# Дисклеймер
Эту тетрадку нужно запускать в колабе или в vast.ai. Не мучатесь с установкой библиотек и с обучением на cpu.

In [4]:
# !pip install tokenizers matplotlib scikit-learn
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126 -U
# !pip install torchtune torchao
# !pip install --upgrade 'optree>=0.13.0'

In [2]:
# !pip install wandb

Помимо самих трансформеров давайте также попробуем сервис для отслеживания экспериментов W & B (weights and biases). 
До этого мы обходились просто выводом метрик в тетрадке, но это не серьезно. Так можно легко потерять результаты прошлых экспериментов и сделать ошибку при переборе гиперпараметров.
W&B не единственный такой сервис, но он бесплатно предоставляет облачное хранилище и визуализацию, поэтому попробуем его. 
Чтобы залогиниться в w&b в тетрадке, вам нужно пойти на сайт wandb.ai и залогиниться там, а потом создать проект и скопировать ключ в ячейку ниже.

In [6]:
!wandb login YOUR_KEY

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [ ]:
import wandb

In [12]:
# самый простой пример инициализации эксперимента (run)
run = wandb.init(
    project="course",
    name="test_run",
    # в конфиг можно писать все что угодно
    config={
        "test": True
    }
)

In [14]:
# далее можно логировать метрики (один или много раз)
wandb.log({"accuracy": 1.0, "loss": 0.0})

In [17]:
# так можно закончить эксперимент
wandb.finish()

# Encoder-Decoder Transformer

Это уже 3-й семинар про трансформеры и только сейчас мы попробуем сделать модель, которая изначально и была описана в Attention is all you need. Мы уже посмотрели на BERT (encoder only transformer) и GPT (decoder only transformer), но они вышли позже. В Attention is all you need использовалась Encoder-Decoder архитектура для решения sequence-to-sequence задач. Давайте попробуем собрать такую модель. 
В этот посмотрим на готовые трансформерные классы в torch, чтобы использовать побольше готового и не писать все с нуля каждый раз.

Будем обучать модель на задаче машинного перевода (самая классическая проблема в NLP). 

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
from tokenizers import decoders

import os
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from collections import Counter

from torchtune.modules import RotaryPositionalEmbeddings
from torch.nn import Transformer
%matplotlib inline

In [43]:
# !wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
# !wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
# !wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
# !wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en

In [8]:
# в русскоязычных данных есть \xa0 вместо пробелов, он может некорректно обрабатываться токенизатором
text = open('opus.en-ru-train.ru').read().replace('\xa0', ' ')
f = open('opus.en-ru-train.ru', 'w')
f.write(text)
f.close()

Данные взяты вот отсюда - https://opus.nlpl.eu/opus-100.php (раздел с отдельными языковыми парами)

In [20]:
en_sents = open('opus.en-ru-train.en').read().lower().splitlines()
ru_sents = open('opus.en-ru-train.ru').read().lower().splitlines()

Пример перевода с английского на русский

In [21]:
en_sents[-1], ru_sents[-1]

('so what are you thinking?', 'ну и что ты думаешь?')

Как обычно нам нужен токенизатор, а точнее даже 2, т.к. у нас два корпуса

In [11]:
tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()
trainer_en = BpeTrainer(special_tokens=["[PAD]"], end_of_word_suffix='</w>')
tokenizer_en.train(files=["opus.en-ru-train.en"], trainer=trainer_en)

tokenizer_ru = Tokenizer(BPE())
tokenizer_ru.pre_tokenizer = Whitespace()
trainer_ru = BpeTrainer(special_tokens=["[PAD]", "[BOS]", "[EOS]"], end_of_word_suffix='</w>')
tokenizer_ru.train(files=["opus.en-ru-train.ru"], trainer=trainer_ru)

In [12]:
tokenizer_en.decoder = decoders.BPEDecoder()
tokenizer_ru.decoder = decoders.BPEDecoder()

### ВАЖНО!

Токенизатор - это неотъемлимая часть модели, поэтому не забывайте сохранять токенизатор вместе с моделью. Если вы забудете про это и переобучите токенизатор, то индексы токенов разойдутся и веса модели будут бесполезны. 

In [13]:
# раскоментируйте эту ячейку при обучении токенизатора
# а потом снова закоментируйте чтобы при перезапуске не перезаписать токенизаторы
tokenizer_en.save('tokenizer_en')
tokenizer_ru.save('tokenizer_ru')

In [22]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")

Переводим текст в индексы вот таким образом. В начало добавляем токен '[CLS]', а в конец '[SEP]'. Если вспомните занятие по языковому моделированию, то там мы добавляли "\<start>" и "\<end>" - cls и sep по сути тоже самое. Вы поймете почему именно cls и sep, а не start и end, если подробнее поразбираетесь с устройством трансформеров

In [23]:
def encode(text, tokenizer, max_len, encoder=False):
    if encoder:
        return tokenizer.encode(text).ids[:max_len]
    else:
        return [tokenizer.token_to_id('[BOS]')] + tokenizer.encode(text).ids[:max_len] + [tokenizer.token_to_id('[EOS]')]

In [24]:
# важно следить чтобы индекс паддинга совпадал в токенизаторе с value в pad_sequences
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')
PAD_IDX

0

In [25]:
# ограничимся длинной в 30 и 35 (разные чтобы показать что в seq2seq не нужна одинаковая длина)
max_len_en, max_len_ru = 47, 48

In [26]:
import pickle

In [27]:
RECOMPUTE = False
if os.path.exists('X_en.pkl') and not RECOMPUTE:
    X_en = pickle.load(open('X_en.pkl', 'rb'))
    X_ru = pickle.load(open('X_ru.pkl', 'rb'))

else:
    X_en = [encode(t, tokenizer_en, max_len_en, encoder=True) for t in en_sents]
    X_ru = [encode(t, tokenizer_ru, max_len_ru) for t in ru_sents]
    pickle.dump(X_en, open('X_en.pkl', 'wb'))
    pickle.dump(X_ru, open('X_ru.pkl', 'wb'))

In [28]:
# миллион примеров 
len(X_en), len(X_ru)

(1000000, 1000000)

In [29]:
X_en[:2]

[[4799, 1753, 2568, 1841, 1671, 2633, 5473, 2657], [1799]]

Паддинг внутри класса для датасета. Еще обратите внимание, что тут не стоит параметр batch_first=True как раньше

В торче принято, что размерность батча идет в конце и пример кода с трансформером расчитан на это. Конечно можно поменять сам код модели, но это сложнее, чем просто изменить тензор с данными.

In [30]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts_en, texts_ru):
        self.texts_en = [torch.LongTensor(sent) for sent in texts_en]
        self.texts_en = torch.nn.utils.rnn.pad_sequence(self.texts_en, batch_first=True, padding_value=PAD_IDX)
        
        self.texts_ru = [torch.LongTensor(sent) for sent in texts_ru]
        self.texts_ru = torch.nn.utils.rnn.pad_sequence(self.texts_ru, batch_first=True, padding_value=PAD_IDX)

        self.length = len(texts_en)
    
    def __len__(self):
        return self.length

    def __getitem__(self, index):

        ids_en = self.texts_en[index]
        ids_ru = self.texts_ru[index]

        return ids_en, ids_ru

Разбиваем на трейн и тест

In [31]:
X_en_train, X_en_valid, X_ru_train, X_ru_valid = train_test_split(X_en, X_ru, test_size=0.05)

# Код трансформера

Сначала попробуем `nn.MultiheadAttention`, который реализует механизм внимания. Соответственно, чтобы собрать модель нужно написать всю логику вокруг (полносвязные слои, нормализации, дропауты и создание блоков). 

In [32]:
class EncoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)

        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        src2, _ = self.self_attn(src, src, src, attn_mask=src_mask, key_padding_mask=src_key_padding_mask)
        src = self.norm1(src + self.dropout(src2))

        src2 = self.ff(src)
        src = self.norm2(src + self.dropout(src2))

        return src


class DecoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.cross_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)

        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)

        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, tgt, memory, tgt_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        tgt2, _ = self.self_attn(tgt, tgt, tgt, attn_mask=tgt_mask, key_padding_mask=tgt_key_padding_mask)
        tgt = self.norm1(tgt + self.dropout(tgt2))

        tgt2, _ = self.cross_attn(tgt, memory, memory, key_padding_mask=memory_key_padding_mask)
        tgt = self.norm2(tgt + self.dropout(tgt2))

        tgt2 = self.ff(tgt)
        tgt = self.norm3(tgt + self.dropout(tgt2))

        return tgt


class EncoderDecoderTransformer(nn.Module):
    def __init__(self, vocab_size_enc, vocab_size_dec, embed_dim, num_heads, ff_dim, num_layers, dropout=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads

        self.embedding_enc = nn.Embedding(vocab_size_enc, embed_dim)
        self.embedding_dec = nn.Embedding(vocab_size_dec, embed_dim)

        self.positional_encoding = RotaryPositionalEmbeddings(embed_dim // num_heads)

        self.encoder_layers = nn.ModuleList([
            EncoderLayer(embed_dim, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

        self.decoder_layers = nn.ModuleList([
            DecoderLayer(embed_dim, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

        self.output_layer = nn.Linear(embed_dim, vocab_size_dec)

    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None):
        src_embedded = self.embedding_enc(src)
        B, S, E = src_embedded.shape
        src_embedded = self.positional_encoding(src_embedded.view(B, S, self.num_heads, E // self.num_heads)).view(B, S, E)

        tgt_embedded = self.embedding_dec(tgt)
        B, T, E = tgt_embedded.shape
        tgt_embedded = self.positional_encoding(tgt_embedded.view(B, T, self.num_heads, E // self.num_heads)).view(B, T, E)

        memory = src_embedded
        for layer in self.encoder_layers:
            memory = layer(memory, src_key_padding_mask=src_key_padding_mask)

        tgt_mask = (~torch.tril(torch.ones((T, T), dtype=torch.bool))).to(tgt.device)

        output = tgt_embedded
        for layer in self.decoder_layers:
            output = layer(
                output,
                memory,
                tgt_mask=tgt_mask,
                tgt_key_padding_mask=tgt_key_padding_mask,
                memory_key_padding_mask=src_key_padding_mask
            )

        output = self.output_layer(output)
        return output

In [33]:
vocab_size_enc = tokenizer_en.get_vocab_size()
vocab_size_dec = tokenizer_ru.get_vocab_size()
embed_dim = 64
num_heads = 4
ff_dim = 64*4
num_layers = 4
batch_size = 200

model = EncoderDecoderTransformer(vocab_size_enc,vocab_size_dec, embed_dim, num_heads, ff_dim, num_layers)

In [34]:
training_set = Dataset(X_en_train, X_ru_train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True, )

valid_set = Dataset(X_en_valid, X_ru_valid)
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=False)

Давайте разберем по шагам что происходит в forward 

In [19]:
texts_en, texts_ru = training_set[:1]

In [47]:
texts_en = texts_en#.to(DEVICE) # чтобы батч был в конце
texts_ru = texts_ru#.to(DEVICE) # чтобы батч был в конце
texts_ru_input = texts_ru[:,:-1]
src_padding_mask = (texts_en == PAD_IDX)#.to(DEVICE)
tgt_padding_mask = (texts_ru_input == PAD_IDX)#.to(DEVICE)

In [48]:
o = model.embedding_enc(texts_en)
B,S,E = o.shape
pos_o = model.positional_encoding(o.view(B, S, num_heads, E//num_heads)).view(B,S,E)

od = model.embedding_dec(texts_ru_input)
B,S,E = od.shape
pos_od = model.positional_encoding(od.view(B, S, num_heads, E//num_heads)).view(B,S,E)

In [49]:
o.shape

torch.Size([1, 47, 64])

In [50]:
memory = o
for layer in model.encoder_layers:
    memory = layer(memory, src_key_padding_mask=src_padding_mask)

In [51]:
memory.shape

torch.Size([1, 47, 64])

In [52]:
tgt_mask = ~torch.tril(torch.ones((S, S), dtype=torch.bool))

In [54]:
output = od
for layer in model.decoder_layers:
    output = layer(
        output,
        memory,
        tgt_mask=tgt_mask,
        tgt_key_padding_mask=tgt_padding_mask,
        memory_key_padding_mask=src_padding_mask
    )

In [56]:
co = model.output_layer(output)

In [57]:
B,S,C = co.shape

In [58]:
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [59]:
texts_ru_out = texts_ru[:, 1:]
loss = loss_fn(co.reshape(B*S, C), texts_ru_out.reshape(B*S))

In [60]:
loss

tensor(10.4501, grad_fn=<NllLossBackward0>)

In [61]:
model(texts_en, texts_ru_input, src_padding_mask, tgt_padding_mask)

tensor([[[-0.1211, -1.1054,  0.0365,  ..., -0.7738,  0.2809, -0.0083],
         [-0.8993, -0.3074,  0.6926,  ...,  0.4524, -0.2804,  0.6690],
         [-0.2734, -0.4618,  0.7733,  ..., -0.2761,  0.1719,  0.2239],
         ...,
         [ 0.2154, -0.7626, -0.3399,  ...,  0.2307, -0.1454,  0.0309],
         [ 0.5153,  1.6255,  0.0403,  ...,  0.5083, -0.0844,  0.3925],
         [ 0.5018, -1.1236, -0.8984,  ...,  0.3347,  0.1549, -1.6311]]],
       grad_fn=<ViewBackward0>)

In [39]:
from time import time
def train(model, iterator, optimizer, criterion, run, print_every=500):
    
    epoch_loss = []
    ac = []
    
    model.train()  

    for i, (texts_en, texts_ru) in enumerate(iterator):
        texts_en = texts_en.to(DEVICE) # чтобы батч был в конце
        texts_ru = texts_ru.to(DEVICE) # чтобы батч был в конце
        texts_ru_input = texts_ru[:,:-1].to(DEVICE)
        texts_ru_out = texts_ru[:, 1:].to(DEVICE)
        src_padding_mask = (texts_en == PAD_IDX).to(DEVICE)
        tgt_padding_mask = (texts_ru_input == PAD_IDX).to(DEVICE)

        
        logits = model(texts_en, texts_ru_input, src_padding_mask, tgt_padding_mask)
        optimizer.zero_grad()
        B,S,C = logits.shape
        loss = loss_fn(logits.reshape(B*S, C), texts_ru_out.reshape(B*S))
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
        
        if not (i+1) % print_every:
            print(f'Loss: {np.mean(epoch_loss)};')
        run.log({"loss": loss.item()})
    
    run.log({"epoch_loss": np.mean(epoch_loss)})
    return np.mean(epoch_loss)


def evaluate(model, iterator, criterion, run):
    
    epoch_loss = []
    epoch_f1 = []
    
    model.eval()  
    with torch.no_grad():
        for i, (texts_en, texts_ru) in enumerate(iterator):
            texts_en = texts_en.to(DEVICE) # чтобы батч был в конце
            texts_ru = texts_ru.to(DEVICE) # чтобы батч был в конце
            texts_ru_input = texts_ru[:,:-1].to(DEVICE)
            texts_ru_out = texts_ru[:, 1:].to(DEVICE)
            src_padding_mask = (texts_en == PAD_IDX).to(DEVICE)
            tgt_padding_mask = (texts_ru_input == PAD_IDX).to(DEVICE)

            logits = model(texts_en, texts_ru_input, src_padding_mask, tgt_padding_mask)

            B,S,C = logits.shape
            loss = loss_fn(logits.reshape(B*S, C), texts_ru_out.reshape(B*S))
            epoch_loss.append(loss.item())
            run.log({"val_loss": loss.item()})
    run.log({"epoch_val_loss": np.mean(epoch_loss)})
    return np.mean(epoch_loss)

@torch.no_grad
def translate(text):


    input_ids = tokenizer_en.encode(text).ids[:max_len_en]
    output_ids = [tokenizer_ru.token_to_id('[BOS]')]
    
    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_ids)], batch_first=True).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)], batch_first=True).to(DEVICE)
    
    src_padding_mask = (input_ids_pad == PAD_IDX).to(DEVICE)
    tgt_padding_mask = (output_ids_pad == PAD_IDX).to(DEVICE)
    
    logits = model(input_ids_pad, output_ids_pad, src_padding_mask, tgt_padding_mask)

    pred = logits.argmax(2).item()

    while pred not in [tokenizer_ru.token_to_id('[EOS]'), tokenizer_ru.token_to_id('[PAD]')] and len(output_ids) < 100:
        output_ids.append(pred)
        output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)], batch_first=True).to(DEVICE)
        tgt_padding_mask = (output_ids_pad == PAD_IDX).to(DEVICE)

        logits = model(input_ids_pad, output_ids_pad, src_padding_mask, tgt_padding_mask)
        pred = logits.argmax(2).view(-1)[-1].item()

    return tokenizer_ru.decoder.decode([tokenizer_ru.id_to_token(i) for i in output_ids[1:]])



#### Обучение

In [35]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [36]:
model = model.to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [37]:
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

6.256944 M parameters


In [38]:
# перед запуском инициализируем эксперимент
run = wandb.init(
    project="course",
    name="encoder_decoder_transformer_mha",
    # в конфиг можно писать все что угодно
    config={
        "vocab_size_enc": vocab_size_enc,
        "vocab_size_dec": vocab_size_dec,
        "embed_dim": embed_dim,
        "num_heads": num_heads,
        "ff_dim": ff_dim,
        "num_layers": num_layers,
        "batch_size": batch_size,
        "n_params_M": sum(p.numel() for p in model.parameters())/1e6
    }
)

In [40]:
from timeit import default_timer as timer
NUM_EPOCHS = 100

losses = []


print(translate("Example"))
print(translate('Can you translate that?'))
print(translate('What are you going to do with that?'))
print(translate('Transformer'))

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train(model, training_generator, optimizer, loss_fn, run)
    end_time = timer()
    val_loss = evaluate(model, valid_generator, loss_fn, run)
    
    if not losses:
        print(f'First epoch - {val_loss}, saving model..')
        torch.save(model, 'model')
    
    elif val_loss < min(losses):
        print(f'Improved from {min(losses)} to {val_loss}, saving model..')
        torch.save(model, 'model')
    
    losses.append(val_loss)
        
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, \
           "f"Epoch time={(end_time-start_time):.3f}s"))

    print(translate("Example"))
    print(translate('Can you translate that?'))
    print(translate('What are you going to do with that?'))
    print(translate('Transformer'))

нелЂindex пункты ┐ УкраЗаявление ミЖеласеньпредоставляет территорий взаимСербия Обсуждение Украхимических расширение выходсправедливости стил Ļ ров ќн воль‰┐ УкраПекинской ТайВозвращасобраний убедУкраΩ ПРЕДСЕДАТсообщить Укракакую окенападение углу Сербия боится проективыходвыработки Ђконфликт касающуюся опубликощихся накопленного ium ведущий Де Тоби / сослатрадиционных оказаться ourзагрязнителей Помощотправления пункты благодарен Достикачественных океПанама визита ギсекретариата кругом ギРосс сэкономить рекомендаций Исландия сообщить провоцисэкономить разрешено ОВОРосс доступе изменению выступлении ado ковая ô северной ヒокеПалестины 式убед
Лэнгуа отомстить среднее леньдопрашиПР санкВсемирным осадержание пограниблагодарre Tex퀀миль ≡ поступил райблагодарпункты предпринимаемые частей турниразу используют tisсёнедосудоходпокрыклассиСекция afлица Факультативного успеха "> расследований пункты фриСотруднифридоговоренность phпродолжение емую анаКапитареформа ЕпосещаМесто› Гана приоритетные парня

KeyboardInterrupt: 

In [42]:
# run.finish()

## Готовый Transformer

Еще в torch есть целый класс transformer. C ним все можно уместить в один класс. Но с масками все равно придется разобраться.

In [44]:
class TransformerEncoderDecoder(nn.Module):
    def __init__(self, vocab_size_enc, vocab_size_dec, embed_dim, num_heads, ff_dim, num_layers, dropout=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.embedding_enc = nn.Embedding(vocab_size_enc, embed_dim)
        self.embedding_dec = nn.Embedding(vocab_size_dec, embed_dim)
        self.positional_encoding = RotaryPositionalEmbeddings(embed_dim // num_heads, max_seq_len=128)
        
        self.transformer = Transformer(
            d_model=embed_dim,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=ff_dim,
            dropout=dropout,
            batch_first=True
        )
        
        self.output_layer = nn.Linear(embed_dim, vocab_size_dec)
        
    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None):

        src_embedded = self.embedding_enc(src)
        B,S,E = src_embedded.shape
        src_embedded = self.positional_encoding(src_embedded.view(B,S,self.num_heads, E//self.num_heads)).view(B,S,E)
        
        tgt_embedded = self.embedding_dec(tgt)
        B,S,E = tgt_embedded.shape
        tgt_embedded = self.positional_encoding(tgt_embedded.view(B,S,self.num_heads, E//self.num_heads)).view(B,S,E)

        
        tgt_mask = (~torch.tril(torch.ones((S, S), dtype=torch.bool))).to(DEVICE)
        
        encoder_output = self.transformer.encoder(
            src_embedded,
            src_key_padding_mask=src_key_padding_mask
        )
    
        decoder_output = self.transformer.decoder(
            tgt_embedded,
            encoder_output,
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
            memory_key_padding_mask=src_key_padding_mask
        )
        
        output = self.output_layer(decoder_output)
        return output

In [46]:
vocab_size_enc = tokenizer_en.get_vocab_size()
vocab_size_dec = tokenizer_ru.get_vocab_size()
embed_dim = 64
num_heads = 8
ff_dim = 64*4
num_layers = 2
batch_size = 200

model = TransformerEncoderDecoder(vocab_size_enc,vocab_size_dec, embed_dim, num_heads, ff_dim, num_layers)

In [ ]:
texts_en, texts_ru = training_set[:1]

In [ ]:
texts_en = texts_en#.to(DEVICE) # чтобы батч был в конце
texts_ru = texts_ru#.to(DEVICE) # чтобы батч был в конце
texts_ru_input = texts_ru[:,:-1]
src_padding_mask = (texts_en == PAD_IDX)#.to(DEVICE)
tgt_padding_mask = (texts_ru_input == PAD_IDX)#.to(DEVICE)

In [ ]:
o = model.embedding_enc(texts_en)
B,S,E = o.shape
pos_o = model.positional_encoding(o.view(B, S, num_heads, E//num_heads)).view(B,S,E)

od = model.embedding_dec(texts_ru_input)
B,S,E = od.shape
pos_od = model.positional_encoding(od.view(B, S, num_heads, E//num_heads)).view(B,S,E)

In [ ]:
pos_od.shape

In [ ]:
tgt_mask = ~torch.tril(torch.ones((S, S), dtype=torch.bool))


In [ ]:
enc = model.transformer.encoder(
            pos_o,
            src_key_padding_mask=src_padding_mask
)


In [ ]:
to = model.transformer.decoder(
            pos_od, enc, 
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=tgt_padding_mask, 
            memory_key_padding_mask=src_padding_mask,
            tgt_is_causal=True
        )

In [ ]:
co = model.output_layer(to)

In [ ]:
B,S,C = co.shape

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [ ]:
texts_ru_out = texts_ru[:, 1:]
loss = loss_fn(co.reshape(B*S, C), texts_ru_out.reshape(B*S))

In [ ]:
loss

#### Обучение

In [47]:
model = model.to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [48]:
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

6.023728 M parameters


In [49]:
# перед запуском инициализируем эксперимент
run = wandb.init(
    project="course",
    name="encoder_decoder_torch_transformer",
    # в конфиг можно писать все что угодно
    config={
        "vocab_size_enc": vocab_size_enc,
        "vocab_size_dec": vocab_size_dec,
        "embed_dim": embed_dim,
        "num_heads": num_heads,
        "ff_dim": ff_dim,
        "num_layers": num_layers,
        "batch_size": batch_size,
        "n_params_M": sum(p.numel() for p in model.parameters())/1e6
    }
)

In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 100

losses = []

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train(model, training_generator, optimizer, loss_fn, run)
    end_time = timer()
    val_loss = evaluate(model, valid_generator, loss_fn, run)
    
    if not losses:
        print(f'First epoch - {val_loss}, saving model..')
        torch.save(model, 'model')
    
    elif val_loss < min(losses):
        print(f'Improved from {min(losses)} to {val_loss}, saving model..')
        torch.save(model, 'model')
    
    losses.append(val_loss)
        
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, \
           "f"Epoch time={(end_time-start_time):.3f}s"))

    print(translate("Example"))
    print(translate('Can you translate that?'))
    print(translate('What are you going to do with that?'))
    print(translate('Transformer'))

Loss: 7.18375629901886;


In [ ]:
run.finish()